In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [4]:
#load trained,scaler pickle and onehat pickle
model=load_model('model.h5')

#load encoder and scaler,
with open('label_encode_gender.pkl','rb') as file:
    label_encode_gender=pickle.load(file)


with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

with open('onehot_geo.pkl','rb') as file:
    onehot_geo=pickle.load(file)


In [19]:
# Example input data
input_data ={
'CreditScore': 600,
'Geography':'France',
'Gender':'Male',
'Age':40,
'Tenure':3,
'Balance':60000,
'NumOfProducts': 2,
'HasCrCard':1,
'IsActiveMember' :1,
'EstimatedSalary' :50000
}

In [20]:
geo_encoded= onehot_geo.transform([[input_data['Geography']]]) .toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_geo.get_feature_names_out(['Geography']))

c:\Users\rathi\OneDrive\Desktop\ann\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [21]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [22]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [23]:
#encode categorical variables
input_df['Gender']=label_encode_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [24]:
#concat with one hot encoded 

input_df=pd.concat([input_df.drop('Geography',axis=1,errors='ignore'),geo_encoded_df],axis=1)



In [25]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [26]:
#scaling
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [28]:
#prediction
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 202ms/step


array([[0.02756699]], dtype=float32)

In [29]:
prediction_proba=prediction[0][0]

In [30]:
if (prediction_proba > 0.5):
    print( 'The customer is likely to churn.')

else:
    print( 'The customer is not likely to churn.')





The customer is not likely to churn.
